# Expectation Maximization (EM) Algorithm

In [2]:
## SETUP
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)

np.set_printoptions(formatter={'all':lambda x: '%.3f' % x})

from IPython.display import Image
from numpy.core.umath_tests import matrix_multiply as mm

from scipy.optimize import minimize
from scipy.stats import bernoulli, binom


/Users/tomereldor/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
#solving for likelihood using minimization
#https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html

#define a log likelihood function
def neg_loglik(thetas, n, xs, zs):
    return -np.sum([binom(n, thetas[z]).logpmf(x) for (x, z) in zip(xs, zs)])

## Complete Information

In [4]:
#defining initial parameters
m = 10
theta_A = 0.8 #weight of coin A
theta_B = 0.3 #weight of coin B
theta_0 = [theta_A, theta_B]

#generating bernouli trials
coin_A = bernoulli(theta_A)
coin_B = bernoulli(theta_B)

#generating dataset of coin tosses according to their weights
xs = map(sum, [coin_A.rvs(m), coin_A.rvs(m), coin_B.rvs(m), coin_A.rvs(m), coin_B.rvs(m)])


In [5]:
## COMPLETE INFORAMTION

# Z's: Which coins (complete information)
zs = [0, 0, 1, 0, 1] #which coin did we toss

## NUMERICAL ESTIMATE, WITH COMPLETE INFORMATION
bnds = [(0,1), (0,1)]
minimize(neg_loglik, [0.5, 0.5], args=(m, xs, zs),
         bounds=bnds, method='tnc', options={'maxiter': 100})

     fun: 7.6552677541114456
     jac: array([0.000, -0.000])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 19
     nit: 7
  status: 1
 success: True
       x: array([0.733, 0.100])

## Incomplete Information

In [18]:
heads = [14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34]
throws = [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]

xs = np.array(zip(heads,throws))
thetas = np.array([[0.2, 0.8], [0.7, 0.3]])


tol = 0.005
max_iter = 100

ll_old = 0
for i in range(max_iter):
    ws_A = []
    ws_B = []

    vs_A = []
    vs_B = []

    ll_new = 0

    # E-step: calculate probability distributions over possible completions
    for x in xs:

        # multinomial (binomial) log likelihood
        ll_A = np.sum([x*np.log(thetas[0])])
        ll_B = np.sum([x*np.log(thetas[1])])

        # [EQN 1]
        denom = np.exp(ll_A) + np.exp(ll_B)
        w_A = np.exp(ll_A)/denom
        w_B = np.exp(ll_B)/denom

        ws_A.append(w_A)
        ws_B.append(w_B)

        # used for calculating theta
        vs_A.append(np.dot(w_A, x))
        vs_B.append(np.dot(w_B, x))

        # update complete log likelihood
        ll_new += w_A * ll_A + w_B * ll_B

    # M-step: update values for parameters given current distribution
    # [EQN 2]
    thetas[0] = np.sum(vs_A, 0)/np.sum(vs_A)
    thetas[1] = np.sum(vs_B, 0)/np.sum(vs_B)
    # print distribution of z for each x and current parameter estimate

    print "Iteration: %d" % (i+1)
    print "theta_A = %.2f, theta_B = %.2f, ll = %.2f" % (thetas[0,0], thetas[1,0], ll_new)

    if np.abs(ll_new - ll_old) < tol:
        break
    ll_old = ll_new

Iteration: 1
theta_A = 0.31, theta_B = 0.44, ll = -744.14
Iteration: 2
theta_A = 0.26, theta_B = 0.41, ll = -692.48
Iteration: 3
theta_A = 0.24, theta_B = 0.40, ll = -684.41
Iteration: 4
theta_A = 0.24, theta_B = 0.40, ll = -683.42
Iteration: 5
theta_A = 0.24, theta_B = 0.40, ll = -683.37
Iteration: 6
theta_A = 0.24, theta_B = 0.40, ll = -683.37


In [7]:

def em(xs, thetas, max_iter=100, tol=1e-6):
    #Expectation-maximization for coin sample problem

    ll_old = -np.infty
    for i in range(max_iter):
        ll = np.array([np.sum(xs * np.log(theta), axis=1) for theta in thetas])
        lik = np.exp(ll)
        ws = lik/lik.sum(0)
        vs = np.array([w[:, None] * xs for w in ws])
        thetas = np.array([v.sum(0)/v.sum() for v in vs])
        ll_new = np.sum([w*l for w, l in zip(ws, ll)])
        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new
    return i, thetas, ll_new


In [8]:
xs = np.array([(5,5), (9,1), (8,2), (4,6), (7,3)])
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])

#run the EM function
i, thetas, ll = em(xs, thetas)
print i
for theta in thetas:
    print theta
print ll
np.random.seed(1234)



18
[0.797 0.203]
[0.520 0.480]
-29.868676155


In [9]:
# generate new parameters and randomized data and test EM function
n = 100
p0 = 0.8
p1 = 0.35
xs = np.concatenate([np.random.binomial(n, p0, n/2), np.random.binomial(n, p1, n/2)])
xs = np.column_stack([xs, n-xs])
print xs
np.random.shuffle(xs)

results = [em(xs, np.random.random((2,2))) for i in range(10)]
i, thetas, ll =  sorted(results, key=lambda x: x[-1])[-1]
print "iteration ", i
for theta in thetas:
    print "ø: ", theta
print "log-likelihood: ", ll

[[84.000 16.000]
 [79.000 21.000]
 [81.000 19.000]
 [77.000 23.000]
 [77.000 23.000]
 [82.000 18.000]
 [82.000 18.000]
 [77.000 23.000]
 [73.000 27.000]
 [75.000 25.000]
 [82.000 18.000]
 [80.000 20.000]
 [78.000 22.000]
 [78.000 22.000]
 [81.000 19.000]
 [79.000 21.000]
 [80.000 20.000]
 [88.000 12.000]
 [77.000 23.000]
 [75.000 25.000]
 [81.000 19.000]
 [79.000 21.000]
 [86.000 14.000]
 [81.000 19.000]
 [74.000 26.000]
 [79.000 21.000]
 [81.000 19.000]
 [77.000 23.000]
 [82.000 18.000]
 [79.000 21.000]
 [75.000 25.000]
 [81.000 19.000]
 [77.000 23.000]
 [84.000 16.000]
 [78.000 22.000]
 [78.000 22.000]
 [83.000 17.000]
 [74.000 26.000]
 [81.000 19.000]
 [75.000 25.000]
 [86.000 14.000]
 [84.000 16.000]
 [86.000 14.000]
 [78.000 22.000]
 [79.000 21.000]
 [80.000 20.000]
 [87.000 13.000]
 [79.000 21.000]
 [82.000 18.000]
 [80.000 20.000]
 [33.000 67.000]
 [34.000 66.000]
 [43.000 57.000]
 [36.000 64.000]
 [30.000 70.000]
 [42.000 58.000]
 [36.000 64.000]
 [30.000 70.000]
 [33.000 67.00

In [13]:
# generate new parameters and randomized data and test EM function
heads = [14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34]
throws = [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]


xs = zip(throws,heads)
np.array(xs)

array([[41.000, 14.000],
       [43.000, 33.000],
       [23.000, 19.000],
       [23.000, 10.000],
       [1.000, 0.000],
       [23.000, 17.000],
       [36.000, 24.000],
       [37.000, 17.000],
       [2.000, 1.000],
       [131.000, 36.000],
       [5.000, 5.000],
       [29.000, 6.000],
       [13.000, 5.000],
       [47.000, 13.000],
       [10.000, 4.000],
       [58.000, 35.000],
       [15.000, 5.000],
       [14.000, 5.000],
       [100.000, 74.000],
       [113.000, 34.000]])

In [14]:
# generate new parameters and randomized data and test EM function
heads = [14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34]
throws = [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]


xs = np.array(zip(heads,throws))

n = 100
#initial ps
p0 = heads[0]/throws[0]
p1 = heads[1]/throws[1]
#np.random.shuffle(xs)

results = [em(xs, np.random.random((2,2))) for i in range(10)]
i, thetas, ll =  sorted(results, key=lambda x: x[-1])[-1]
print "iteration ", i
for theta in thetas:
    print "ø: ", theta
print "log-likelihood: ", ll

iteration  9
ø:  [0.235 0.765]
ø:  [0.401 0.599]
log-likelihood:  -683.367176614


In [16]:
### Formatted as a function: adapted from : https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html
def em_gmm_orig(xs, pis, mus, sigmas, tol=0.01, max_iter=100):

    n, p = xs.shape
    k = len(pis)

    ll_old = 0
    for i in range(max_iter):
        exp_A = []
        exp_B = []
        ll_new = 0

        # E-step
        ws = np.zeros((k, n))
        for j in range(len(mus)):
            for i in range(n):
                ws[j, i] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
        ws /= ws.sum(0)

        # M-step
        pis = np.zeros(k)
        for j in range(len(mus)):
            for i in range(n):
                pis[j] += ws[j, i]
        pis /= n

        mus = np.zeros((k, p))
        for j in range(k):
            for i in range(n):
                mus[j] += ws[j, i] * xs[i]
            mus[j] /= ws[j, :].sum()

        sigmas = np.zeros((k, p, p))
        for j in range(k):
            for i in range(n):
                ys = np.reshape(xs[i]- mus[j], (2,1))
                sigmas[j] += ws[j, i] * np.dot(ys, ys.T)
            sigmas[j] /= ws[j,:].sum()

        # update complete log likelihoood
        ll_new = 0.0
        for i in range(n):
            s = 0
            for j in range(k):
                s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
            ll_new += np.log(s)

        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new

    return ll_new, pis, mus, sigmas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import Axes3D
from sklearn import mixture

def q(x, y):
	g1 = mlab.bivariate_normal(x, y, 1.0, 1.0, -1, -1, -0.8)
	g2 = mlab.bivariate_normal(x, y, 1.5, 0.8, 1, 2, 0.6)
	return 0.6*g1+28.4*g2/(0.6+28.4)

def plot_q():
	fig = plt.figure()
	ax = fig.gca(projection='3d')
	X = np.arange(-5, 5, 0.1)
	Y = np.arange(-5, 5, 0.1)
	X, Y = np.meshgrid(X, Y)
	Z = q(X, Y)
	surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=plt.get_cmap('coolwarm'),
			linewidth=0, antialiased=True)
	fig.colorbar(surf, shrink=0.5, aspect=5)

	plt.savefig('3dgauss.png')
	plt.clf()

def sample():
	'''Metropolis Hastings'''
	N = 10000
	s = 10
	r = np.zeros(2)
	p = q(r[0], r[1])
	print p
	samples = []
	for i in xrange(N):
		rn = r + np.random.normal(size=2)
		pn = q(rn[0], rn[1])
		if pn >= p:
			p = pn
			r = rn
		else:
			u = np.random.rand()
			if u < pn/p:
				p = pn
				r = rn
		if i % s == 0:
			samples.append(r)

	samples = np.array(samples)
	plt.scatter(samples[:, 0], samples[:, 1], alpha=0.5, s=1)

	'''Plot target'''
	dx = 0.01
	x = np.arange(np.min(samples), np.max(samples), dx)
	y = np.arange(np.min(samples), np.max(samples), dx)
	X, Y = np.meshgrid(x, y)
	Z = q(X, Y)
	CS = plt.contour(X, Y, Z, 10, alpha=0.5)
	plt.clabel(CS, inline=1, fontsize=10)
	plt.savefig("samples.png")
	return samples

def fit_samples(samples):
	gmix = mixture.GMM(n_components=2, covariance_type='full')
	gmix.fit(samples)
	print gmix.means_
	colors = ['r' if i==0 else 'g' for i in gmix.predict(samples)]
	ax = plt.gca()
	ax.scatter(samples[:,0], samples[:,1], c=colors, alpha=0.8)
	plt.savefig("class.png")

if __name__ == '__main__':
	plot_q()
	s = sample()
	fit_samples(s)

/Users/tomereldor/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


0.00632453617818


/Users/tomereldor/anaconda/lib/python2.7/site-packages/sklearn/utils/deprecation.py:57: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tomereldor/anaconda/lib/python2.7/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tomereldor/anaconda/lib/python2.7/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/tomereldor/anaconda/lib/python2.7/site-packages/sklearn/

[[ 0.89502149  1.99249233]
 [-0.93856387 -1.09068318]]
